In [1]:
import pandas as pd
import numpy as np
from dataAccessor import get_intraday, get_daily, get_singletime
# from config import av_api_key
from util import compare_df_cols, TIME_MAP

In [33]:
class backTester():

    
    
    def __init__(self, order_book = None, stat_mode = None):
        self.orders = pd.DataFrame(columns=['order_id','ticker','order_type','exec_date','exec_time','exec_fulldate'])
        self.stat_mode = stat_mode

        self.MAX_TRADE_DAYS = 180
        
        if order_book: 
            self.add_orders(order_book)
            
    def add_orders(self,order_book):

        order_book['exec_fulldate'] = order_book['exec_date'] + order_book['exec_time'].map(TIME_MAP)
       
        # adds orders to order book
        print(order_book.head())
        a_not_b = compare_df_cols(self.orders, order_book)
        assert a_not_b.empty
        
        to_add = order_book[['order_id','order_type','exec_time','exec_date','ticker','exec_fulldate']]
        self.orders = pd.concat([self.orders, order_book])

    def get_pfolio_exec(self, stat_mode):
        
        self.orders['exec_price']=self.orders.apply(
            lambda row: get_singletime(
                ticker=row['ticker'],date=row['exec_date'],dot=row['exec_time'],price_freq=stat_mode
            )['price'].values[0], 
            axis=1
        )

    def get_pfolio_fwd(self, stat_mode):
        if stat_mode == 'daily':

            # get forward prices of each order's ticker starting at exec date
            self.orders['fwd'] = self.orders.apply(
                lambda row: get_daily(
                    row['ticker'],
                    start_date=row['exec_date']
                    ,end_date=row['exec_date']+pd.Timedelta(days=self.MAX_TRADE_DAYS)
                ), 
                axis = 1
            )

            # unnest, creates df of prices with associated order_id on each row
            fwd_prices = (
                pd.concat(self.orders.set_index('order_id').pop('fwd').to_dict())
                .rename_axis(('order_id','new'))
                .reset_index(level=0)
                .reset_index(drop=True)
            )

            # changes open/close columns into one column called trade_price
            fwd_prices_open = fwd_prices[['order_id','date','open']]
            fwd_prices_open.loc[:,'date'] = fwd_prices_open['date'] + pd.Timedelta(hours=9,minutes=30)
            fwd_prices_open.rename(columns={'date':'fwd_date','open':'fwd_price'}, inplace=True)

            fwd_prices_close = fwd_prices[['order_id','date','close']]
            fwd_prices_close.loc[:,'date'] = fwd_prices_close['date'] + pd.Timedelta(hours=16)
            fwd_prices_close.rename(columns={'date':'fwd_date','close':'fwd_price'}, inplace=True)
            
            self.pfolio_prices = pd.concat([fwd_prices_open, fwd_prices_close])
            
        elif stat_mode == 'intraday':
            raise NotImplementedError('TODO')
    
    def trade(self, stat_mode = None):
        if stat_mode:
            self.stat_mode = stat_mode
        # get data
        if stat_mode == 'daily':
            # get exec prices
            self.get_pfolio_exec(self.stat_mode)
            # get forward prices
            self.get_pfolio_fwd(self.stat_mode)
        
        elif stat_mode == 'intraday':
            raise NotImplementedError('TODO')
        # calculate positions

        # generate pfolio stats

    def results(self):
        raise NotImplementedError('TODO')


In [34]:
b = backTester()

In [35]:
test_orders = pd.DataFrame(
    [
        [1,'buy','open',pd.to_datetime('2023-08-04'),'IBM'],
        [2,'buy','open',pd.to_datetime('2022-03-08'),'IBM']
    ], 
    index = [1, 2],
    columns=['order_id','order_type','exec_time','exec_date','ticker']
)
test_orders

,order_id,order_type,exec_time,exec_date,ticker
1,1,buy,open,2023-08-04,IBM
2,2,buy,open,2022-03-08,IBM


In [36]:
b.add_orders(test_orders)

   order_id order_type exec_time  exec_date ticker       exec_fulldate
1         1        buy      open 2023-08-04    IBM 2023-08-04 09:30:00
2         2        buy      open 2022-03-08    IBM 2022-03-08 09:30:00


In [37]:
b.orders

,order_id,ticker,order_type,exec_date,exec_time,exec_fulldate
1,1,IBM,buy,2023-08-04,open,2023-08-04 09:30:00
2,2,IBM,buy,2022-03-08,open,2022-03-08 09:30:00


In [38]:
b.trade(stat_mode = 'daily')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


C:\Users\danie\AppData\Local\Temp\ipykernel_12216\2530425180.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fwd_prices_open.rename(columns={'date':'fwd_date','open':'fwd_price'}, inplace=True)
C:\Users\danie\AppData\Local\Temp\ipykernel_12216\2530425180.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fwd_prices_close.rename(columns={'date':'fwd_date','close':'fwd_price'}, inplace=True)


In [39]:
b.orders

,order_id,ticker,order_type,exec_date,exec_time,exec_fulldate,exec_price,fwd
1,1,IBM,buy,2023-08-04,open,2023-08-04 09:30:00,145.089996,date open high ...
2,2,IBM,buy,2022-03-08,open,2022-03-08 09:30:00,126.620003,date open high ...


In [40]:
b.pfolio_prices

,order_id,fwd_date,fwd_price
0,1,2023-08-04 09:30:00,145.089996
1,1,2023-08-07 09:30:00,145.000000
2,1,2023-08-08 09:30:00,145.699997
3,1,2023-08-09 09:30:00,144.940002
4,1,2023-08-10 09:30:00,143.039993
...,...,...,...
131,2,2022-08-29 16:00:00,130.309998
132,2,2022-08-30 16:00:00,129.580002
133,2,2022-08-31 16:00:00,128.449997
134,2,2022-09-01 16:00:00,129.660004
